In [1]:
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pickle as pk
import os
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

In [2]:
data_dir = "./mimic-iii"
files_dir = "./dump_files"

In [14]:
notes = pd.read_csv(os.path.join(data_dir, 'NOTEEVENTS.csv'))
patients = pd.read_csv(os.path.join(data_dir, 'PATIENTS.csv'),parse_dates=[3],date_parser=lambda x:datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
diag = pd.read_csv(os.path.join(data_dir, 'DIAGNOSES_ICD.csv'))

/home/akshara/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
diag

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [16]:
diag['HADM_ID'].nunique()

58976

In [17]:
diag['ICD9_CODE'] = diag.ICD9_CODE.astype(str)
diag = diag.drop(['ROW_ID','SEQ_NUM','SUBJECT_ID'],axis = 1)
diag = pd.pivot_table(diag,index='HADM_ID',values='ICD9_CODE',aggfunc=list)

In [18]:
mlb = MultiLabelBinarizer()

In [19]:
df1 = pd.DataFrame(mlb.fit_transform(diag['ICD9_CODE']),columns=mlb.classes_, index=diag.index)
df1.reset_index(level=df1.index.names, inplace=True)
df1 = df1.drop('nan', axis=1)

In [21]:
df1

,HADM_ID,0030,0031,0038,0039,0041,0048,0049,0051,00581,...,V8801,V8811,V8812,V8821,V9010,V902,V9039,V9081,V9089,V9103
0,100001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,100003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100006,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,100007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,100009,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,199993,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58972,199994,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58973,199995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58974,199998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
df1.to_csv(os.path.join(files_dir, "hadm_icd.csv"), index=False)

## load hadm-icd file

In [25]:
df1 = pd.read_csv(os.path.join(files_dir, "hadm_icd.csv"))

In [15]:
l = df1.sum(axis=0).tolist()

In [16]:
l

[8844678466,
 2,
 1,
 2,
 1,
 3,
 1,
 1,
 1,
 2,
 1,
 1,
 7,
 1,
 1,
 1,
 1444,
 1,
 4,
 1,
 2,
 6,
 17,
 139,
 26,
 19,
 5,
 4,
 2,
 1,
 1,
 1,
 1,
 8,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 10,
 4,
 1,
 1,
 15,
 2,
 16,
 2,
 4,
 2,
 1,
 2,
 1,
 16,
 3,
 5,
 5,
 1,
 376,
 44,
 515,
 118,
 150,
 88,
 110,
 60,
 4,
 467,
 127,
 27,
 395,
 206,
 3725,
 7,
 1,
 3,
 10,
 25,
 2,
 26,
 37,
 81,
 4,
 614,
 4,
 144,
 58,
 703,
 239,
 293,
 13,
 473,
 967,
 191,
 39,
 213,
 404,
 12,
 10,
 52,
 300,
 204,
 71,
 19,
 538,
 1,
 5,
 1,
 1,
 3,
 29,
 5,
 1,
 1,
 3,
 12,
 3,
 5,
 4,
 7,
 6,
 2,
 2,
 3,
 1,
 46,
 2,
 5,
 2,
 7,
 73,
 1,
 27,
 2,
 2,
 7,
 32,
 21,
 1,
 1,
 2,
 1,
 3,
 27,
 114,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 18,
 21,
 19,
 173,
 5,
 290,
 2,
 14,
 174,
 1,
 93,
 1,
 1,
 1218,
 2,
 674,
 69,
 1,
 11,
 2,
 2,
 1,
 6,
 7,
 5,
 91,
 2,
 3,
 2,
 2,
 7,
 1,
 11,
 1,
 16,
 1,
 79,
 4,
 2,
 1,
 4,
 2,
 1,
 1,
 1,
 4,
 1,
 23,
 1

In [38]:
## Notes

In [4]:
notes = pd.read_csv(os.path.join(data_dir, 'NOTEEVENTS.csv'))

/home/akshara/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/akshara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/akshara/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
snowBallStemmer = SnowballStemmer("english")
exclude = list(nltk.corpus.stopwords.words("english"))
exclude.extend(list(string.punctuation))

def remove_stopwords(text):
    text = re.sub('\[\*\*[^\]]*\*\*\]', ' ', text)
    text = re.sub('<[^>]*>', ' ', text)
    text = re.sub('[\W\_]+', ' ', text.lower()) 
    text = re.sub(' \d+', ' ', text)
    return " ".join([word for word in nltk.tokenize.word_tokenize(text) if word not in exclude])

notes['TEXT'] = notes['TEXT'].apply(str)        
notes["TEXT"] = notes["TEXT"].apply(lambda x: remove_stopwords(x))

In [7]:
def preprocess_text(df):
    df.TEXT = df.TEXT.fillna(' ')
    df.TEXT = df.TEXT.str.replace('\n',' ')
    df.TEXT = df.TEXT.str.replace('\r',' ')
    df.TEXT = df.TEXT.str.replace(' +', ' ')
    return df

df_text = preprocess_text(notes)

In [9]:
df_text['CATEGORY'].unique()

array(['Discharge summary', 'Echo', 'ECG', 'Nursing', 'Physician ',
       'Rehab Services', 'Case Management ', 'Respiratory ', 'Nutrition',
       'General', 'Social Work', 'Pharmacy', 'Consult', 'Radiology',
       'Nursing/other'], dtype=object)

In [167]:
#df_text = notes

In [17]:
notes_rad = df_text[df_text['CATEGORY']=='Radiology']
notes_disch = df_text[df_text['CATEGORY']=='Discharge summary']
notes_nurse = df_text[df_text['CATEGORY']=='Nursing']
notes_gen = df_text[df_text['CATEGORY']=='General']
notes_phys = df_text[df_text['CATEGORY']=='Physician ']

In [9]:
print(f'RAD: {notes_rad.shape}\nNURSE: {notes_nurse.shape}\nDISCH: {notes_disch.shape}\nPHYS: {notes_phys.shape}')

RAD: (45526, 2)
NURSE: (9070, 2)
DISCH: (52726, 2)
PHYS: (8983, 2)


In [156]:
notes_rad['HADM_ID'].nunique()

45526

In [12]:
notes_rad = notes_rad.groupby('HADM_ID')['TEXT'].apply(' '.join).reset_index()
notes_disch = notes_disch.groupby('HADM_ID')['TEXT'].apply(' '.join).reset_index()
notes_phys = notes_phys.groupby('HADM_ID')['TEXT'].apply(' '.join).reset_index()
notes_nurse = notes_nurse.groupby('HADM_ID')['TEXT'].apply(' '.join).reset_index()

In [13]:
notes_phys

,HADM_ID,TEXT
0,100003.0,chief complaint coffee ground emesis light hea...
1,100018.0,sicu hpi male w extensive medical history incl...
2,100020.0,chief complaint hypotension saw examined patie...
3,100037.0,chief complaint hour events received units pla...
4,100063.0,chief complaint intoxication seizure hpi hx et...
...,...,...
8978,199909.0,tsicu hpi yom p stairs etoh tx osh imaging sho...
8979,199911.0,chief complaint melena coffee ground emesis sa...
8980,199940.0,sicu hpi pod7 yom p l fem pt situ svg ef cr wt...
8981,199948.0,chief complaint respiratory distress saw exami...


In [ ]:
# merge inner

In [10]:
df_all1 = notes_rad.merge(notes_disch, on='HADM_ID')

In [14]:
df_all1['NOTES'] = df_all1['TEXT_x'].map(str) + ' ' + df_all1['TEXT_y'].map(str)

In [18]:
df_all1 = df_all1[['HADM_ID', 'NOTES']]

In [22]:
df_all2

,HADM_ID,NOTES
0,100003.0,59 year old male hcv related cirrhosis grade i...
1,100018.0,impaired physical mobility assessment patient ...
2,100020.0,58yom withl progressive ms since baclofen abdo...
3,100037.0,chief complaint pancytopenia ich hpi presented...
4,100063.0,pt male found lying outside liquor store intox...
...,...,...
8950,199909.0,pt log roll precautions hob degrees remains ne...
8951,199911.0,52 yo f history hcv cirrhosis grade varices mu...
8952,199940.0,demographics attending md r admit diagnosis no...
8953,199948.0,74 f contributory pmh presented rectal prolaps...


In [11]:
df_all2 = notes_nurse.merge(notes_phys, on='HADM_ID')

In [20]:
df_all2['NOTES'] = df_all2['TEXT_x'].map(str) + ' ' + df_all2['TEXT_y'].map(str)

In [21]:
df_all2 = df_all2[['HADM_ID', 'NOTES']]

In [ ]:
for i in range(1):
    for j in [xx]:#[notes_rad, notes_disch, notes_nurse, notes_phys]:
        print(j.iloc[i].TEXT)
        print('\n')
        print(j.iloc[i].TRANS_TEXT)

    print('\n')

In [14]:
notes_rad.to_csv(os.path.join(files_dir, 'notes_rad.csv'), index=False)
notes_disch.to_csv(os.path.join(files_dir, 'notes_disch.csv'), index=False)
notes_nurse.to_csv(os.path.join(files_dir, 'notes_nurse.csv'), index=False)
notes_phys.to_csv(os.path.join(files_dir, 'notes_phys.csv'), index=False)

In [3]:
# load
notes_rad = pd.read_csv(os.path.join(files_dir, 'notes_rad.csv'))
notes_disch = pd.read_csv(os.path.join(files_dir, 'notes_disch.csv'))
notes_nurse = pd.read_csv(os.path.join(files_dir, 'notes_nurse.csv'))
notes_phys = pd.read_csv(os.path.join(files_dir, 'notes_phys.csv'))

In [121]:
!pip install googletrans

     |████████████████████████████████| 55 kB 970 kB/s eta 0:00:011
     |████████████████████████████████| 42 kB 138 kB/s eta 0:00:011
     |████████████████████████████████| 994 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 3.9 MB/s eta 0:00:011
     |████████████████████████████████| 65 kB 518 kB/s eta 0:00:011
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=2048085c96a43d37dd21c3002c579e2120642de92f328d486f3dc20d2756dd0c
  Stored in directory: /home/akshara/.cache/pip/wheels/0e/ce/9b/d51de1064911d42480ab6b57fc943ee36572441f27546354e2
Successfully built googletrans


In [126]:
notes_rad['TEXT'].isna().sum()

0

In [145]:
!pip install google_trans_new

In [146]:
from google_trans_new import google_translator  

translator = google_translator()  
translate_text = translator.translate('Hola mundo!', lang_src='es', lang_tgt='en')

In [147]:
translate_text

'Hello World! '

In [23]:
notes_rad

,HADM_ID,TEXT
0,100001.0,chest pa lat clip reason eval infiltrate medic...
1,100003.0,pm liver gallbladder us single organ clip reas...
2,100006.0,chest portable ap clip reason please assess in...
3,100009.0,pm chest pa lat clip reason pre op admitting d...
4,100010.0,pm chest portable ap different physician reaso...
...,...,...
45521,199993.0,chest portable ap clip reason r effusion medic...
45522,199994.0,pm chest portable ap clip reason pt self extub...
45523,199995.0,pm abdomen u complete study duplex dopp abd pe...
45524,199998.0,pm chest port line placement clip reason line ...


In [177]:
def trans(text):
    text_ch = text.split('. ')
    text_tr = []
    for x in text_ch:
        x = translator.translate(x, lang_src='en', lang_tgt='es')
        x = translator.translate(x, lang_src='es', lang_tgt='en')
        text_tr.append(x)
    return ". ".join(text_tr)

In [180]:
xx = notes_phys.head(1)

In [ ]:
for df in [xx]:#, notes_disch, notes_nurse, notes_rad]:
    df['TRANS_TEXT_1'] = df['TEXT'].apply(translator.translate, lang_src='en', lang_tgt='es')
    df['TRANS_TEXT'] = df['TRANS_TEXT_1'].apply(translator.translate, lang_src='es', lang_tgt='en')

In [182]:
for df in [xx]:#, notes_disch, notes_nurse, notes_rad]:
    df['TRANS_TEXT'] = df.apply(lambda x: trans(x['TEXT']), axis=1)

<ipython-input-182-46a8156cf769>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TRANS_TEXT'] = df.apply(lambda x: trans(x['TEXT']), axis=1)


## Merge

In [26]:
df_rad = notes_rad.merge(df1, how = 'inner',  left_on="HADM_ID", right_on = "HADM_ID")
df_disch = notes_disch.merge(df1, how = 'inner',  left_on="HADM_ID", right_on = "HADM_ID")
df_nurse = notes_nurse.merge(df1, how = 'inner',  left_on="HADM_ID", right_on = "HADM_ID")
df_phys = notes_phys.merge(df1, how = 'inner',  left_on="HADM_ID", right_on = "HADM_ID")

In [28]:
df_rad_disch = df_all1.merge(df1, how = 'inner',  left_on="HADM_ID", right_on = "HADM_ID")
df_nurse_phys = df_all2.merge(df1, how = 'inner',  left_on="HADM_ID", right_on = "HADM_ID")

In [30]:
df_rad.to_csv(os.path.join(files_dir, 'rad.csv'), index=False)
df_disch.to_csv(os.path.join(files_dir, 'disch.csv'), index=False)
df_nurse.to_csv(os.path.join(files_dir, 'nurse.csv'), index=False)
df_phys.to_csv(os.path.join(files_dir, 'phys.csv'), index=False)

In [31]:
df_rad_disch.to_csv(os.path.join(files_dir, 'rad_disch.csv'), index=False)
df_nurse_phys.to_csv(os.path.join(files_dir, 'nurse_phys.csv'), index=False)

In [118]:
print(f'RAD: {df_rad.shape}\nNURSE: {df_nurse.shape}\nDISCH: {df_disch.shape}\nPHYS: {df_phys.shape}')

RAD: (45526, 6986)
NURSE: (9070, 6986)
DISCH: (52726, 6986)
PHYS: (8983, 6986)


In [194]:
dd = pd.read_csv(os.path.join(files_dir, 'notes_rad.csv'))

In [195]:
dd

,HADM_ID,TEXT
0,100001.0,chest pa lat clip reason eval infiltrate medic...
1,100003.0,pm liver gallbladder us single organ clip reas...
2,100006.0,chest portable ap clip reason please assess in...
3,100009.0,pm chest pa lat clip reason pre op admitting d...
4,100010.0,pm chest portable ap different physician reaso...
...,...,...
45521,199993.0,chest portable ap clip reason r effusion medic...
45522,199994.0,pm chest portable ap clip reason pt self extub...
45523,199995.0,pm abdomen u complete study duplex dopp abd pe...
45524,199998.0,pm chest port line placement clip reason line ...


In [ ]:
'''
## Combine notes based on sim
tf_matrix = tfidf.fit_transform(doc)
scores = []
short_kb = ""
for i in range(kb_vec.shape[0]):
        scores.append({'idx':i, 'sim': cosine_similarity(utt_vec, kb_vec[i])})
    ref = heapq.nlargest(2, scores, key=lambda s: s['sim'])
    del doc[0]
    for i in ref:
        short_kb += doc[i['idx']] + ". "'''